In [1]:
import astropy.units as u
from astropy.time import Time

In [2]:
from nustar_lunar_pointing.orbit import get_epoch_tle

In [14]:
from datetime import datetime
from sgp4.earth_gravity import wgs72
from sgp4.io import twoline2rv
from astropy.io.fits import getdata, getheader

tlefile = '../data/NUSTAR_TLE_ARCHIVE.txt.2016122'
att = getdata('../data/nu30102028006A.attorb')
hdr = getheader('../data/nu30102028006A.attorb', 1)
mjdref = hdr['MJDREFI']
met = att[0]['TIME']*u.s + mjdref*u.d #+ mjdreff*u.d

print(mjdreff)

checktime = Time(met.to(u.d), format = 'mjd').datetime
print(checktime.isoformat())
mindt, line1, line2 = get_epoch_tle(checktime, tlefile)
print(mindt)
print(line1)
print(line2)


satellite = twoline2rv(line1, line2, wgs72)


0.00076601852
2016-04-29T09:35:02
0
1 99105U 12031A   16120.39200650  .00001435  00000-0  79059-4 0  9993
2 99105   6.0268  71.6065 0011540  97.4088 262.7353 14.87347181210735


In [15]:
print(att[0]['POSITION'])

position, velocity = satellite.propagate(
    checktime.year, checktime.month, checktime.day, checktime.hour, checktime.minute, checktime.second)
print(position)

[-2425.53173828  6524.64599609   466.88754272]
(-2464.1505833583865, 6510.350554074878, 463.96566970651753)


In [16]:
from astropy.coordinates import EarthLocation

In [21]:
foo = EarthLocation.from_geocentric(*position * u.km)

In [35]:
print(foo.latitude.deg)
print(foo.longitude.deg)
print(foo.height)
print(att[0]['SAT_LAT'])
print(att[0]['SAT_LON'])
print(att[0]['SAT_ALT'] - u.R_earth.to(u.km))

3.836611870483933
110.73152266062574
598.4876946149096 km
3.83724
108.866
598.409898437


In [39]:
from astropy.coordinates import get_moon
from astropy.time import Time



In [78]:
pos = list(position)
pos[0] += 50
foo = EarthLocation.from_geocentric(*pos * u.km)

In [79]:
t = Time(checktime)
loc = foo
moon_coords = get_moon(t,loc)
ra_moon1, dec_moon1 = moon_coords.ra.degree, moon_coords.dec.degree


In [80]:
# Old version
ra_moon, dec_moon

(301.34952574681444, -16.033258601587445)

In [77]:
# New version
ra_moon, dec_moon

(301.3429546046582, -16.03425571775049)

In [81]:
from astropy.coordinates import SkyCoord

In [85]:
sk1 = SkyCoord(301.34952574681444*u.deg, -16.033258601587445*u.deg)
sk2 = SkyCoord(301.3429546046582*u.deg, -16.03425571775049*u.deg)
sk2 - sk1

TypeError: unsupported operand type(s) for -: 'SkyCoord' and 'SkyCoord'

In [86]:
sk1.separation(sk2)

<Angle 0.006393748592840043 deg>